In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [7]:
pip install pyDOE

In [ ]:
!pip install torchinfo

In [8]:

from argparse import ArgumentParser
from collections import defaultdict
from math import ceil, sin
from math import floor

#from jax.example_libraries import optimizers
#from jax.experimental.ode import odeint

import scipy
import pandas as pd
import numpy as np

import numpy as np
import torch

#import torch_xla
#import torch_xla.core.xla_model as xm
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
from torch.autograd.functional import jacobian
from torch.nn.modules.container import T

from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib as mpl
from functools import partial

In [9]:
from pyDOE import lhs

Run this notebook with cuda

In [ ]:
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device=xm.xla_device()

In [11]:
device

device(type='cuda')

In [12]:
alpha = torch.tensor(0.01)  # Example value for thermal diffusivity
hidden_dim =40
hidden_layers = 6
input_dim=2
output_dim=1
lambda_f=1
resblocks=0 # 0 when not Resnet
activation='gelu'
learningRate=0.0001
wd=1e-4
N=3000
N_f=3000
N_u=1000
n_epochs=6000
network='MLP' # MLP Or Resnet

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Heatfinal",

    # track hyperparameters and run metadata
    config={
    "Thermal_diff": alpha,
    "N_f": N_f,
    "N_u":N_u,
    "learning_rate": learningRate,
    "weight_decay": wd,
    "hidden_layers": hidden_layers,
    "hidden_dim": hidden_dim,
    'input_dim': input_dim,
    'output_dim': output_dim,
    'activation':activation,
    'lambdaB': lambda_f,
    'numResblocks': resblocks,
    "epochs": n_epochs,
    'network': network
    }
)

In [13]:
# Function to plot the the three loss terms initial/boundary condition and residual
def plot_loss(losses):
    fig, ax = plt.subplots()
    fig.canvas.manager.set_window_title("Loss terms")

    # Extract the loss values from the dictionary
    lossNN = losses["Res"]
    lossEQ = losses["BC"]
    lossIC=losses['IC']
    lossData=losses['Data']
    # Plot both losses
    ax.plot(lossNN, label='Residual')
    ax.plot(lossEQ, label='BC')
    ax.plot(lossIC, label='IC')
    #ax.plot(lossData, label='Data')

    ax.legend()
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.set_title("Neural Network and Equation/BC/IC Losses over Epochs")
    plt.show()


In [14]:
losses = defaultdict(list)

In [15]:
# Xavier initialization
def xavier_init(module):
        for m in module.modules():
            if type(m) == nn.Linear:
                nn.init.xavier_uniform_(m.weight)


In [16]:
# Sinusoidal activation function
class SinusActivation(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.sin(x)

In [17]:
# MLP neural network as a function of neurons (hidden_dim), number of layers and activation function
def construct_network(input_dim, output_dim, hidden_dim, hidden_layers, activation):
    # Activation function mapping
    activations = {
        'swish': nn.SiLU,
        'tanh': nn.Tanh,
        'relu': nn.ReLU,
        'gelu': nn.GELU,
        'softplus': nn.Softplus,
        'sin': SinusActivation
    }

    # Check if activation is valid
    if activation not in activations:
        raise ValueError(f"Invalid activation function. Expected one of: {list(activations.keys())}")

    ActivationFunc = activations[activation]

    # Construct the layers
    layers = [nn.Linear(input_dim, hidden_dim), ActivationFunc()]
    for _ in range(hidden_layers):
        layers.extend([nn.Linear(hidden_dim, hidden_dim), ActivationFunc()])
    layers.append(nn.Linear(hidden_dim, output_dim))
    layers.append(ActivationFunc())  # Activation function for the output layer


    # Create the network
    net = nn.Sequential(*layers).double().to(device)

    xavier_init(net)
    return net

# Example usage:
# net = construct_network(10, 2, 128, 3, 'relu', device='cuda')



In [18]:
# Extend the MLP network with a learnable parameter
class PINN(nn.Module):
    def __init__(self, base_network):
        super(PINN, self).__init__()
        self.network = base_network
        self.nu = nn.Parameter(torch.tensor([-5.0], dtype=torch.float64))  # Initialize nu as a learnable parameter
        #self.phi=nn.Parameter(torch.tensor([0.5], dtype=torch.float64))
    def forward(self, x):
        return self.network(x)

In [ ]:
# ResNet neural network as a function of neurons, number resnet block, the size of last layer and the activation function
class ResidualBlockScalar(nn.Module):
    def __init__(self, features, activation_func):
        super(ResidualBlockScalar, self).__init__()
        self.linear1 = nn.Linear(features, features)
        self.linear2 = nn.Linear(features, features)
        self.activation = activation_func()

    def forward(self, x):
        residual = x
        x = self.activation(self.linear1(x))
        x = self.linear2(x)
        return self.activation(x + residual)

class SimpleResNetScalar(nn.Module):
    def __init__(self, num_classes, features, num_blocks, activation='relu'):
        super(SimpleResNetScalar, self).__init__()
        activations = {
            'swish': nn.SiLU,
            'tanh': nn.Tanh,
            'relu': nn.ReLU,
            'gelu': nn.GELU,
            'softplus': nn.Softplus,
            'sin': SinusActivation  # Assuming SinusActivation is defined elsewhere
        }

        self.linear1 = nn.Linear(2, features)
        self.activation = activations[activation]()
        self.res_blocks = nn.ModuleList([ResidualBlockScalar(features, activations[activation]) for _ in range(num_blocks)])
        self.linear2 = nn.Linear(features, num_classes)
        self.final_activation = activations[activation]()  # Activation function after the final layer

    def forward(self, x):
        x = self.activation(self.linear1(x))
        for block in self.res_blocks:
            x = block(x)
        x = self.linear2(x)
        return self.final_activation(x)  # Apply activation function here

# Example instantiation
# model = SimpleResNetScalar(num_classes=output_dim, features=hidden_dim, num_blocks=resblocks, activation='swish')


In [ ]:
print(model)

SimpleResNetScalar(
  (linear1): Linear(in_features=2, out_features=50, bias=True)
  (activation): SiLU()
  (res_blocks): ModuleList()
  (linear2): Linear(in_features=50, out_features=1, bias=True)
  (final_activation): SiLU()
)


In [ ]:
#from torchinfo import summary

In [ ]:
#summary(model)

In [19]:

# Model parameters

# Set random seed for reproducibility
torch.manual_seed(54)
# Interior conditions
x_int=lhs(2,N_f)
x_int=torch.tensor(x_int)
u_int = torch.zeros(N_f, 1)
u_train=torch.exp(-alpha * np.pi**2 * x_int[...,0]) * torch.sin(np.pi * x_int[...,1])

# Initial conditions
x_ini = torch.rand(N_u, 2)
x_ini[..., 0] = 0  # t = 0
u_ini = torch.sin(np.pi * x_ini[..., 1]).reshape(N_u, 1)

# Boundary conditions
x_bc = torch.rand(N_u, 2)  # t, 0 to 1
x_bc[..., 1] = torch.tensor([0, 1])[torch.randint(0, 2, (N_u,))]  # 0 or 1
u_bc = torch.zeros(N_u, 1)

# Merge inputs
x_inp = torch.cat([x_int, x_bc, x_ini], dim=0).float()
u_inp = torch.cat([u_int, u_bc, u_ini], dim=0).float()

# Print shapes to verify
print(f"x_inp shape: {x_inp.shape}, u_inp shape: {u_inp.shape}")
x_bc_inp = x_bc
x_ini_inp = x_ini
#x_inp = x_int
u_bc_inp = u_bc
u_ini_inp = u_ini
#u_inp = u_int

x_inp = torch.reshape(x_inp,(-1,2));
u_inp = torch.reshape(u_inp,(-1,1));

x_bc_inp = torch.reshape(x_bc_inp,(-1,2));
u_bc_inp = torch.reshape(u_bc_inp,(-1,1));

x_ini_inp = torch.reshape(x_ini_inp,(-1,2));
u_ini_inp = torch.reshape(u_ini_inp,(-1,1));


x_inp shape: torch.Size([5000, 2]), u_inp shape: torch.Size([5000, 1])


In [20]:
# Initialize the model
#model = PINN()

base_network = construct_network(input_dim, output_dim, hidden_dim, hidden_layers, activation)
model = PINN(base_network).double().to(device)


#model=construct_network(input_dim, output_dim, hidden_dim, hidden_layers, activation)
#model = SimpleResNetScalar(num_classes=output_dim, features=hidden_dim, num_blocks=resblocks, activation=activation).double().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=wd)


In [21]:
print(model)

PINN(
  (network): Sequential(
    (0): Linear(in_features=2, out_features=40, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): GELU(approximate='none')
    (4): Linear(in_features=40, out_features=40, bias=True)
    (5): GELU(approximate='none')
    (6): Linear(in_features=40, out_features=40, bias=True)
    (7): GELU(approximate='none')
    (8): Linear(in_features=40, out_features=40, bias=True)
    (9): GELU(approximate='none')
    (10): Linear(in_features=40, out_features=40, bias=True)
    (11): GELU(approximate='none')
    (12): Linear(in_features=40, out_features=40, bias=True)
    (13): GELU(approximate='none')
    (14): Linear(in_features=40, out_features=1, bias=True)
    (15): GELU(approximate='none')
  )
)


In [22]:
def data_loss(xint,u_obs):
  u_pred=model(xint.double().to(device))
  #print(u_obs.shape)
  return torch.mean((u_pred.flatten()-u_obs.double().to(device).flatten())**2)

In [23]:
def initial_loss(xinit,uinit):
  # Predictions for initial
  u_ini_pred = model(xinit.double().to(device))
  # return initial loss
  return torch.mean((u_ini_pred.flatten() - uinit.double().to(device).flatten()) ** 2)

In [24]:
def boundary_loss(xbc,ubc):
  # Predictions for boundary conditions
  u_bc_pred = model(xbc.double().to(device))
  # return loss for boundary conditions
  return torch.mean((u_bc_pred.flatten() - ubc.double().to(device).flatten()) ** 2)

In [25]:
def Heat(x_data):
  # Calculate prediction of u dependent on x, t
  x_physics = x_data.requires_grad_(True)
  u = model(x_physics.double().to(device))
  # Calculate the grad of u with respect to x
  grad_u = torch.autograd.grad(u, x_physics, grad_outputs=torch.ones_like(u)
                               , create_graph=True)[0]
  u_t = grad_u[:, 0] # The first derivative of u with respect to t
  u_x = grad_u[:, 1] # The first derivative of u with respect to x
  # Compute the second derivative of u with respect to x
  u_xx = torch.autograd.grad(u_x, x_physics, grad_outputs=torch.ones_like(u_x)
  , create_graph=True)[0][:, 1]
  #alpha_tensor = torch.tensor(alpha)
  pde_residual = u_t - torch.exp(model.nu).to(device)*u_xx
  return torch.mean(pde_residual**2)


In [26]:
loss_history = []
losses={}
losses["Res"] = []
losses["BC"] = []
losses['IC']=[]
losses['Data']=[]


# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()

    # Loss initial condition
    #loss_initial = initial_loss(x_ini_inp, u_ini_inp)
    # loss boundary condition
    #loss_boundary = boundary_loss(x_bc_inp, u_bc_inp)

    # Loss physics equation
    loss_physics = Heat(x_int.to(device))
    loss_data=data_loss(x_int,u_train)
    # Total loss
    total_loss = lambda_f*(loss_physics)+loss_data
    #+loss_initial + loss_boundary
    #+loss_data
    loss_history.append(total_loss.item())
    # Backwadation
    total_loss.backward()
    # Optimize
    optimizer.step()
    losses['Res'].append(loss_physics.item())
    #losses['BC'].append(loss_boundary.item())
    #losses['IC'].append(loss_initial.item())
    #losses['Data'].append(loss_data.item())
    #wandb.log({"lossRES": loss_physics,
    #                   "lossInitial ": loss_initial,
    #                   "LossBoundary": loss_boundary,
    #                    "LossTotal": total_loss})
    if epoch % 100 == 0:
       print(f'Epoch {epoch}, Loss: {total_loss.item()}, Nu:{model.nu.item()} ')



Epoch 0, Loss: 0.3579532021081022, Nu:-4.9999000019999675 
Epoch 100, Loss: 0.1581674847539321, Nu:-4.989904186937837 
Epoch 200, Loss: 0.0869268381722262, Nu:-4.979950856667573 
Epoch 300, Loss: 0.08667941452204364, Nu:-4.969980571298314 
Epoch 400, Loss: 0.08646202861920181, Nu:-4.960005842525172 
Epoch 500, Loss: 0.08625156372179932, Nu:-4.950031840330664 
Epoch 600, Loss: 0.08603011177549177, Nu:-4.940059082577258 
Epoch 700, Loss: 0.08577327301808169, Nu:-4.930086573645976 
Epoch 800, Loss: 0.08544209560824034, Nu:-4.9201115367999355 
Epoch 900, Loss: 0.08496133258601539, Nu:-4.910128151835809 
Epoch 1000, Loss: 0.08415686329063472, Nu:-4.9001238250514785 
Epoch 1100, Loss: 0.08253573497970404, Nu:-4.890067672450586 
Epoch 1200, Loss: 0.07822441598637854, Nu:-4.8798698132904335 
Epoch 1300, Loss: 0.06021178325427562, Nu:-4.869273398063747 
Epoch 1400, Loss: 0.004921595815446752, Nu:-4.874030165651614 
Epoch 1500, Loss: 0.0014277781920840386, Nu:-4.896553089460102 
Epoch 1600, Loss

In [27]:
print('Truth',0.01)
estimate_nu=np.exp(model.nu.item())
print('Estimated alpha',estimate_nu)
print('% error ', np.abs(  (estimate_nu-0.01)/0.01  )*100   )


Truth 0.01
Estimated alpha 0.009549591312279945
% error  4.504086877200556
